In [29]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider
import random
import matplotlib.pyplot as plt 
import math

# Setup provider

In [2]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider (
    resource_id = #please enter your resource id here ,
    location = "eastus"
)

In [4]:
print([backend.name() for backend in provider.backends()])

['ionq.qpu', 'ionq.simulator']


In [5]:
simulator_backend = provider.get_backend("ionq.simulator")

# Quantum Jeopardy

In [38]:
def apply_gates(gates, qc):
    for gate in gates:
        getattr(qc,gate[0])(gate[1])
    return qc


#applying user gates (the user can choose up to three gates to run)
def get_input():
    gate_set = []
    for i in range(3):
        gate_type = input("What gate would you like to use? (Type None if no gate): ")
        if gate_type in ["x", "y", "z", "t", "None"]:
            if gate_type != "None":
                qubit_type = int(input("Which qubit would you like to apply the gate to?: "))
                gate_set.append((gate_type, qubit_type))
            else:
                print("skipped code")
        else:
            raise ValueError("please input a correct gate")
            
    return gate_set

def play_round(make_random = True):
    """
    1. It initializes the circuit in a random state
    2. It calls get_input() to let the user choose which gates they want to apply to the circuit. The user's
    goal is to maximize the probability of choosing a certain topic, represented by a quantum state 
    (|00> -> 0, |01> -> 1, |10> -> 2, |11> -> 3).
    3. The final circuit is run by run_result using either the simulator or quantum hardware
    4. The results are parsed.
    5. The most common measurment is converted into a choice. (|00> -> 0, |01> -> 1, |10> -> 2, |11> -> 3) 
    """
    # circuit initialization (We can set the rotation angles randomly) 
    circuit = QuantumCircuit(2, 2)
    circuit.h(0)
    circuit.cx(0,1) #this creates a bell state |00> + |11> . The qubits are maximaly entangled.
    
    # adding random rotations
    if make_random:
        rotations = [random.random()*2*math.pi for i in range(4)]
        circuit.rx(rotations[0],0)
        circuit.rz(rotations[1],0)
        circuit.rx(rotations[2],1)
        circuit.rz(rotations[3],1)
    
    print(circuit)
    gate_set = get_input()           
    circuit = apply_gates(gate_set, circuit)
    circuit.measure([0, 1], [0, 1])
    print(circuit)
    
    result = run_result(circuit)
    
    # parsing results
    counts = {format(n, "02b"): 0 for n in range(4)}
    counts.update(result.get_counts(circuit))
    print(counts)
    
    # finding which measurment was measured the most amount of times
    max_counts = 0
    max_key = None
    for key,count in counts.items():
        if count > max_counts:
            max_counts = count
            max_key = key
    max_key = max_key[::-1] # reversing string since qiskit represents |01> as "10"
    choice = int(max_key, 2)
    return choice
    

def run_result(qc):
    job = simulator_backend.run(qc, shots=100)
    job_id = job.id()
    print("Job id", job_id)
    job_monitor(job)
    result = job.result()
    return result

In [39]:
play_round()

     ┌───┐     ┌────────────┐┌────────────┐
q_0: ┤ H ├──■──┤ Rx(2.9871) ├┤ Rz(1.4677) ├
     └───┘┌─┴─┐├────────────┤├────────────┤
q_1: ─────┤ X ├┤ Rx(4.0294) ├┤ Rz(5.4535) ├
          └───┘└────────────┘└────────────┘
c: 2/══════════════════════════════════════
                                           


What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  1
What gate would you like to use? (Type None if no gate):  y
Which qubit would you like to apply the gate to?:  0
What gate would you like to use? (Type None if no gate):  t
Which qubit would you like to apply the gate to?:  1


     ┌───┐     ┌────────────┐┌────────────┐┌───┐     ┌─┐   
q_0: ┤ H ├──■──┤ Rx(2.9871) ├┤ Rz(1.4677) ├┤ Y ├─────┤M├───
     └───┘┌─┴─┐├────────────┤├────────────┤├───┤┌───┐└╥┘┌─┐
q_1: ─────┤ X ├┤ Rx(4.0294) ├┤ Rz(5.4535) ├┤ X ├┤ T ├─╫─┤M├
          └───┘└────────────┘└────────────┘└───┘└───┘ ║ └╥┘
c: 2/═════════════════════════════════════════════════╩══╩═
                                                      0  1 
Job id ce29e33c-8196-11ec-92ef-367dda17a95d
Job Status: job has successfully run
{'00': 43, '01': 6, '10': 6, '11': 43}


0

In [40]:
play_round(make_random = False)

     ┌───┐     
q_0: ┤ H ├──■──
     └───┘┌─┴─┐
q_1: ─────┤ X ├
          └───┘
c: 2/══════════
               


What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  0
What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  0
What gate would you like to use? (Type None if no gate):  x
Which qubit would you like to apply the gate to?:  1


     ┌───┐     ┌───┐┌───┐┌─┐
q_0: ┤ H ├──■──┤ X ├┤ X ├┤M├
     └───┘┌─┴─┐├───┤└┬─┬┘└╥┘
q_1: ─────┤ X ├┤ X ├─┤M├──╫─
          └───┘└───┘ └╥┘  ║ 
c: 2/═════════════════╩═══╩═
                      1   0 
Job id 1fbc7494-8197-11ec-92ef-367dda17a95d
Job Status: job has successfully run
{'00': 0, '01': 50, '10': 50, '11': 0}


2